# HOME ASSIGNMENT #3: SLACK API - TO GSHEET
**Mục đích của bài Assignment**
- Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
- `**[Optional 1]**` Đưa danh sách Users lên Google Spreadsheet, để theo dõi 
- `**[Optional 2]**` Lấy thông tin Assignment Submission và số Reviews trên `#atom-assignmentnt2` và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng**
- Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
- Trích xuất thông tin từ JSON
- Dùng module gspread để đưa thông tin lên Google Spreadsheet

## 0. Load Modules

In [2]:
pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install gspread_dataframe

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install oauth2client

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

## 1. Slack API: User List
* Bạn có thể đọc lại về concept API [HERE](https://anhdang.gitbook.io/datacracy/atom/3-data-tools-2/3.2-spotify-api-and-postman)
* Assignment này sẽ dùng Slack API để lấy thông tin về Learners và theo dõi các bài tập đã nộp và được review (sau đó cập nhật lên Google Spreadsheet)
* ===> **NOTICE**: Slack API authorize bằng Bearer Token `xoxb-...-...-...` (Sẽ được cung cấp riêng)
* Update file `env_variable.json` như trong [Assignment#2](../assignment_2/home_assignment_2.ipynb)
* ==> Nếu bạn dùng Google Colab, upload file vào Colab ([Hướng dẫn](https://colab.research.google.com/notebooks/io.ipynb))

In [2]:
os.listdir()

['env_variable.json',
 'home_assignment_3.ipynb',
 'README.md',
 'week-4-test-python-20-05-21-f8187e392724.json']

In [3]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [4]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 

In [5]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

In [7]:
user_dat[0:1]

[{'id': 'USLACKBOT',
  'team_id': 'T01B7SGGMLJ',
  'name': 'slackbot',
  'deleted': False,
  'color': '757575',
  'real_name': 'Slackbot',
  'tz': 'America/Los_Angeles',
  'tz_label': 'Pacific Daylight Time',
  'tz_offset': -25200,
  'profile': {'title': '',
   'phone': '',
   'skype': '',
   'real_name': 'Slackbot',
   'real_name_normalized': 'Slackbot',
   'display_name': 'Slackbot',
   'display_name_normalized': 'Slackbot',
   'fields': None,
   'status_text': '',
   'status_emoji': '',
   'status_expiration': 0,
   'avatar_hash': 'sv41d8cd98f0',
   'always_active': True,
   'first_name': 'slackbot',
   'last_name': '',
   'image_24': 'https://a.slack-edge.com/80588/img/slackbot_24.png',
   'image_32': 'https://a.slack-edge.com/80588/img/slackbot_32.png',
   'image_48': 'https://a.slack-edge.com/80588/img/slackbot_48.png',
   'image_72': 'https://a.slack-edge.com/80588/img/slackbot_72.png',
   'image_192': 'https://a.slack-edge.com/80588/marketing/img/avatars/slackbot/avatar-slackbo

### TODO #1
Hoàn tất đoạn code sau

In [8]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [9]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [10]:
user_df[user_df.display_name == 'Anh Hà Văn Huỳnh'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
57,U01VB632FPS,huynhanh.ha94,Anh Hà Văn Huỳnh,Anh Ha Van Huỳnh,,,False


-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

### TODO#2
Tạo service account (output là file json), file này để cho phép ta access vào Google Spreadsheet:

1. Làm theo hướng dẫn: [Google Create a Service Account](https://support.google.com/a/answer/7378726?hl=en)
![google_service_account](../img/google_service_account.png)
2. Lưu file JSON (chứa credential về máy)
![gservice_acc_json](../img/gservice_acc_json.png)
3. Nhớ Enable [Google Drive API](https://console.cloud.google.com/marketplace/product/google/drive.googleapis.com?q=search&referrer=search&project=quickstart-313303) (Nếu bạn chạy code báo lỗi chưa enable API thì vào link trong phần lỗi để Enable, sau khi kích hoạt có thể cần vài phút để chạy được)
![enable_api](../img/enable_api.png)
* ==> Upload file Gsheet Credential JSON nếu bạn dùng Colab 
* ==> Nếu bạn để key trong repo git, **NHỚ** để file json vào `.gitignore` để không bị leaked key)


In [11]:
os.listdir()

['env_variable.json',
 'home_assignment_3.ipynb',
 'README.md',
 'week-4-test-python-20-05-21-f8187e392724.json']

In [12]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'week-4-test-python-20-05-21-f8187e392724.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


**Tạo Spreadsheet**

1. Tạo Spreadsheet trên google
2. Invite account trong `client_email` (file JSON Gsheet Credential bên trên) vào Spreadsheet (quyền Editor)
![enable_api](../img/enable_api.png)
3. Lấy `SPREADSHEET_KEY` (nằm trong chính URL của Spreadhstee): `https://docs.google.com/spreadsheets/d/<SPREADSHEET_KEY>/edit#gid=0`

![add_gsheet](../img/add_gsheet.png)

In [78]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '15kOg9fSJlHgqNc2_96leIaryJm_dDqz8PMbpN9IpTc0' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

![slack_user_gsheet](../img/slack_user_gsheet.png)

-------------- HẾT PHẦN OPTION 1 ---------------------

## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [14]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [15]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'id': 'C01B4PVGLVB',
 'name': 'general',
 'is_channel': True,
 'is_group': False,
 'is_im': False,
 'created': 1600856703,
 'is_archived': False,
 'is_general': True,
 'unlinked': 0,
 'name_normalized': 'general',
 'is_shared': False,
 'parent_conversation': None,
 'creator': 'U01BE2PR6LU',
 'is_ext_shared': False,
 'is_org_shared': False,
 'shared_team_ids': ['T01B7SGGMLJ'],
 'pending_shared': [],
 'pending_connected_team_ids': [],
 'is_pending_ext_shared': False,
 'is_member': False,
 'is_private': False,
 'is_mpim': False,
 'topic': {'value': '', 'creator': '', 'last_set': 0},
 'purpose': {'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.',
  'creator': 'U01BE2PR6LU',
  'last_set': 1600856703},
 'previous_names': [],
 'num_members': 63}

### TODO#3 
* Tìm id của channel #atom-assignment2

In [25]:
# Tìm tên đúng của các channel
for i in channel_ls:
    print(i['name'])

general
contents
branding-design
atom-assignment1
atom-week1
atom-general
topics-data-analytics
topics-python
topics-materials
atom-assignment2
atom-week2
atom-week3
atom-assignment3


In [26]:
# Tim id của #atom-assignment2 (được thể hiện với tên 'atom-assignment2')
for i in channel_ls:
    if i['name'] == 'atom-assignment2':
        print(i['id'])


C021FSDN7LJ


### Slack API: List messages trong 1 channel

In [16]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [17]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [18]:
msg_ls[21]

{'client_msg_id': '65967e82-ee13-4c5d-8cd8-8cfec7050f5e',
 'type': 'message',
 'text': 'Chào mọi người, mình  gửi Assignment 1 nhé:\n<https://github.com/AnhThuNguyenHuynh/datacracy-atom-NguyenHuynhAnhThu>',
 'user': 'U01VB63LRNC',
 'ts': '1619229687.043200',
 'team': 'T01B7SGGMLJ',
 'edited': {'user': 'U01VB63LRNC', 'ts': '1619233440.000000'},
 'attachments': [{'fallback': 'AnhThuNguyenHuynh/datacracy-atom-NguyenHuynhAnhThu',
   'title': 'AnhThuNguyenHuynh/datacracy-atom-NguyenHuynhAnhThu',
   'id': 1,
   'color': '24292f',
   'fields': [{'title': 'Language',
     'value': 'Jupyter Notebook',
     'short': True},
    {'title': 'Last updated', 'value': '8 minutes ago', 'short': True}],
   'bot_id': 'B01UQU8E2A3',
   'app_unfurl_url': 'https://github.com/AnhThuNguyenHuynh/datacracy-atom-NguyenHuynhAnhThu',
   'is_app_unfurl': True}],
 'blocks': [{'type': 'rich_text',
   'block_id': 'Tjy',
   'elements': [{'type': 'rich_text_section',
     'elements': [{'type': 'text',
       'text': 'Chà

In [27]:
not_learners_id = ['U01BE2PR6LU']

In [28]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      print(ts, user, reply_users_count, reply_users, latest_reply, github)

2021-05-10 11:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 16:14:31.003600 https://github.com/danhpcv/
2021-05-10 11:35:39.003800 U01UMC08KL2 2 ['U01UTGS5ZNY', 'U01UMC08KL2'] 2021-05-18 15:09:59.000800 https://github.com/auslynnguyen/
2021-05-08 13:31:59.005200 U01UMJ36QD9 1 ['U01UMBX2X18'] 2021-05-08 13:36:43.006000 https://github.com/auslynnguyen/
2021-05-04 20:55:46.001800 U01UTGVPE7N 1 ['U01UMBX2X18'] 2021-05-08 13:36:43.006000 https://github.com/hoaintp/
2021-05-04 00:41:18.002400 U01U6JQB695 2 ['U01UMC0N5U2', 'U01U6JQB695'] 2021-05-11 10:10:06.001300 https://github.com/Tenderriver/
2021-05-03 15:51:19.001200 U01V00KHHHP 1 ['U01UMBZHU2W'] 2021-05-15 00:51:26.024600 https://github.com/saturn1101/


### TODO#4
* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

In [29]:
# Lấy dữ liệu messages từ channel assignment#2
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C021FSDN7LJ"} ## This is ID of assignment#2 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [30]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_as2 = response_json['messages']

In [54]:
#Test xem dữ liệu của 'msg_as2'
msg_as2[34].keys()

dict_keys(['client_msg_id', 'type', 'text', 'user', 'ts', 'team', 'blocks', 'thread_ts', 'reply_count', 'reply_users_count', 'latest_reply', 'reply_users', 'is_locked', 'subscribed', 'reactions'])

In [68]:
as2_dict = {'ts':[], 'user':[], 'reply_users_count':[], 'reply_count':[],'reply_users':[],'latest_reply':[],'github':[]}

In [69]:
# Tạo bảng
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
## github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in msg_as2:
    if i['user'] not in not_learners_id:
      ts = dt.fromtimestamp(float(i['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
      as2_dict['ts'].append(ts)
      as2_dict['user'].append(i['user']) # -> Lấy thông tin người post messages
      text = i['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
    
      if len(github_link) > 0: 
          as2_dict['github'].append(github_link[0])
      else: 
          as2_dict['github'].append('NA')
      if 'reply_count' in i.keys(): 
          as2_dict['reply_count'].append(i['reply_count']) #-> Extract số review, Se bi nhieu vi co nhieu comment hong phai de nhan xet
      else: 
          as2_dict['reply_count'].append('NA')
      if 'reply_users_count' in i.keys(): 
          as2_dict['reply_users_count'].append(i['reply_users_count']) #
      else:
          as2_dict['reply_users_count'].append('NA')
      if 'reply_users' in i.keys(): 
          as2_dict['reply_users'].append(i['reply_users'])
      else: 
          as2_dict['reply_users'].append('NA')
      if 'latest_reply' in i.keys(): 
          latest_reply = dt.fromtimestamp(float(i['latest_reply']))
          as2_dict['latest_reply'].append(latest_reply)
      else: 
          as2_dict['latest_reply'].append('NA')

In [71]:
len(as2_dict['user'])

56

In [72]:
as2_data = pd.DataFrame(as2_dict)
as2_data.head(3)

,ts,user,reply_users_count,reply_count,reply_users,latest_reply,github
0,2021-05-20 08:58:15.008600,U0226MQ73MJ,NA,NA,NA,NA,NA
1,2021-05-20 07:19:34.008100,U01UMBX2X18,NA,NA,NA,NA,https://github.com/thienphu71/
2,2021-05-19 21:53:45.005800,U01USGKQ771,NA,NA,NA,NA,https://github.com/TrangNgo1709/


In [81]:
# ACCES GOOGLE SHEET
sheet_index_no = 1
spreadsheet_key = '15kOg9fSJlHgqNc2_96leIaryJm_dDqz8PMbpN9IpTc0' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet1 = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet1, as2_data) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

-------------- HẾT PHẦN OPTION 2 ---------------------